In [32]:
!pip install keras-tuner

In [33]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [34]:
dataset = keras.datasets.fashion_mnist

In [35]:
(train_images,train_labels),(test_images,test_labels) = dataset.load_data()

In [36]:
train_images = train_images/255
test_images = test_images/255

In [37]:
train_images.shape

(60000, 28, 28)

In [38]:
train_labels.shape

(60000,)

In [39]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [40]:
train_images.shape

(60000, 28, 28, 1)

In [41]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [42]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [43]:
tuner_search = RandomSearch(build_model,objective = 'val_accuracy',max_trials = 5,directory = 'output',project_name = 'Demo')

INFO:tensorflow:Reloading Oracle from existing project output/Demo/oracle.json
INFO:tensorflow:Reloading Tuner from output/Demo/tuner0.json


In [44]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [45]:
model=tuner_search.get_best_models(num_models=1)[0]

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        34608     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,639,914
Trainable params: 2,639,914
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1592 - accuracy: 0.9391 - val_loss: 0.2532 - val_accuracy: 0.9135
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1179 - accuracy: 0.9549 - val_loss: 0.2799 - val_accuracy: 0.9145
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0897 - accuracy: 0.9668 - val_loss: 0.3211 - val_accuracy: 0.9098
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0660 - accuracy: 0.9754 - val_loss: 0.3668 - val_accuracy: 0.9122
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0511 - accuracy: 0.9814 - val_loss: 0.4073 - val_accuracy: 0.9120
Epoch 9/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0409 - accuracy: 0.9855 - val_loss: 0.4837 - val_accuracy: 0.9102
Epoch 10/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0330 - accuracy: 0.9878 - val_loss: 0.4722 - val_a

In [48]:
pred = model.predict(test_images)

In [54]:
pred.shape

(10000, 10)

In [55]:
test_labels.shape

(10000,)

In [57]:
pred

array([[4.1096734e-11, 9.6364400e-24, 1.3132767e-12, ..., 1.5813249e-12,
        9.7978670e-24, 1.0000000e+00],
       [9.1527568e-09, 1.4068607e-17, 9.9999988e-01, ..., 5.0793899e-14,
        9.4766618e-16, 7.0692863e-10],
       [2.1381155e-32, 1.0000000e+00, 2.0644423e-32, ..., 0.0000000e+00,
        2.0467136e-24, 6.2615319e-34],
       ...,
       [3.8240898e-29, 1.8501791e-33, 3.8417819e-29, ..., 8.7880537e-30,
        1.0000000e+00, 1.8014265e-27],
       [4.1603867e-33, 1.0000000e+00, 1.5409051e-36, ..., 0.0000000e+00,
        3.3471068e-36, 0.0000000e+00],
       [2.5460017e-10, 3.8877519e-11, 1.2241931e-13, ..., 4.6415405e-08,
        4.0905282e-04, 5.0575913e-08]], dtype=float32)

In [58]:
labels = np.argmax(pred, axis=-1)

In [50]:
!pip install sklearn

In [60]:
from sklearn.metrics import r2_score
print(r2_score(labels,test_labels))

0.8362370422258449
